In [0]:
sc

SparkContext 

 Spark UI 

 
 Version 
 v3.2.1 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
print(spark)

In [0]:
# a) Create Airline Pysaprk sql dataFrame. 

airplane_DF = spark.read.csv("dbfs:/FileStore/tables/Flights_Delay.csv",header=True,inferSchema = True)
airplane_DF.show()


+---+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
| ID|YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|
+---+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+-----------

In [0]:
# b) Describe the table schema & show 10 rows of Dataset
airplane_DF.printSchema()
airplane_DF.show(10, truncate=False)

root
 |-- ID: integer (nullable = true)
 |-- YEAR: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAY: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- FLIGHT_NUMBER: integer (nullable = true)
 |-- TAIL_NUMBER: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- SCHEDULED_DEPARTURE: integer (nullable = true)
 |-- DEPARTURE_TIME: integer (nullable = true)
 |-- DEPARTURE_DELAY: integer (nullable = true)
 |-- TAXI_OUT: integer (nullable = true)
 |-- WHEELS_OFF: integer (nullable = true)
 |-- SCHEDULED_TIME: integer (nullable = true)
 |-- ELAPSED_TIME: integer (nullable = true)
 |-- AIR_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- WHEELS_ON: integer (nullable = true)
 |-- TAXI_IN: integer (nullable = true)
 |-- SCHEDULED_ARRIVAL: integer (nullable = true)
 |-- ARRIVAL_TIME: integer (nullable = true

In [0]:
from pyspark.sql.functions import*

In [0]:
airplane_DF.createOrReplaceTempView("planetable")

In [0]:
# c) Average arrival delay caused by airlines.

arr_delay=spark.sql("select AIRLINE,AVG(ARRIVAL_DELAY) as Avg_ARRIVAL_DELAY from planetable group by AIRLINE order by Avg_ARRIVAL_DELAY")
arr_delay.show()

+-------+------------------+
|AIRLINE| Avg_ARRIVAL_DELAY|
+-------+------------------+
|     AS|-1.531766200762389|
|     DL|2.8144726712856043|
|     WN| 3.697840458351697|
|     HA| 4.072423398328691|
|     VX| 5.128571428571429|
|     US| 5.977315185481719|
|     UA| 6.697221614526362|
|     AA| 8.386631979187513|
|     OO|10.154792043399638|
|     EV|10.884270870655678|
|     B6| 13.95852534562212|
|     NK|14.206426484907498|
|     MQ|19.231592604605904|
|     F9|24.103448275862068|
+-------+------------------+



In [0]:
# d) Days of months with average Arrival delays
day_delay=spark.sql("select DAY,AVG(ARRIVAL_DELAY) as Avg_ARRIVAL_DELAY from planetable group by DAY order by Avg_ARRIVAL_DELAY")
day_delay.show()

+---+--------------------+
|DAY|   Avg_ARRIVAL_DELAY|
+---+--------------------+
| 31|  -1.196594427244582|
| 10|-0.04705882352941...|
| 29| 0.07971014492753623|
| 14|  1.3299319727891157|
| 19|  1.6344282238442822|
|  7|  2.8309417040358746|
| 15|   2.966753585397653|
| 28|   3.257425742574257|
| 13|  3.3769751693002257|
| 18|  3.5693430656934306|
| 20|  3.8770149253731345|
| 11|  3.9912935323383083|
| 23|   4.207086133170434|
|  9|   4.421887390959556|
| 30|   4.471478463329452|
| 27|   4.706711409395973|
| 25|   4.903708523096942|
|  8|   5.232349165596919|
| 24|   5.737543859649123|
| 22|   6.550920245398773|
+---+--------------------+
only showing top 20 rows



In [0]:
#e) Days of months with median Departure delays.
median = spark.sql("select DAY, percentile_approx(DEPARTURE_DELAY, 0.5) as median_dep_delay from planetable group by DAY").show()
#airplane_DF.groupBy('DAY').mean('DEPARTURE_DELAY').show()

+---+----------------+
|DAY|median_dep_delay|
+---+----------------+
| 31|              -3|
| 28|              -2|
| 27|              -2|
| 26|              -1|
| 12|               0|
| 22|              -1|
|  1|               0|
| 13|              -1|
|  6|               0|
| 16|              -1|
|  3|               1|
| 20|              -2|
|  5|               1|
| 19|              -2|
| 15|              -1|
|  9|              -1|
| 17|               0|
|  4|               0|
|  8|              -1|
| 23|              -1|
+---+----------------+
only showing top 20 rows



In [0]:
# f) Arrange weekdays with respect to the average arrival delays.
#airplane_DF.groupBy('DAY_OF_WEEK').mean('ARRIVAL_DELAY').show()
spark.sql("select DAY_OF_WEEK, AVG(ARRIVAL_DELAY) as ARRIVAL_DELAY from planetable group by DAY_OF_WEEK order by ARRIVAL_DELAY").show()

+-----------+------------------+
|DAY_OF_WEEK|     ARRIVAL_DELAY|
+-----------+------------------+
|          6| 4.888689138576779|
|          3| 5.587079407806191|
|          5| 6.010538373424971|
|          4| 7.174969021065675|
|          2| 8.033644102148358|
|          7|10.110840438489646|
|          1|10.807447207297264|
+-----------+------------------+



In [0]:
# g) Show Analysis for each month with total number of cancellations. (Descending).
#data1=plane_DF.groupBy('MONTH').agg({'CANCELLED' : 'sum'})
#data1.sort(col('sum(CANCELLED)').desc()).show()
cancel=spark.sql("select MONTH,SUM(CANCELLED) as Total_cancelled from planetable group by MONTH order by SUM(CANCELLED)")
cancel.show()

+-----+---------------+
|MONTH|Total_cancelled|
+-----+---------------+
|    3|            456|
|    1|            608|
|    2|           1087|
+-----+---------------+



In [0]:
# h) Show the top 10 busiest airport. [Hint: Sum Total of Arrival and Departures from an Airport].
#plane_DF.groupBy('ORIGIN_AIRPORT').agg({'DESTINATION_AIRPORT' : 'sum'})
Busiest_airport = spark.sql("select ORIGIN_AIRPORT, SUM(DEPARTURE_TIME + ARRIVAL_TIME) as Busiest_airport from planetable group by ORIGIN_AIRPORT order by Busiest_airport desc").show(10)


+--------------+---------------+
|ORIGIN_AIRPORT|Busiest_airport|
+--------------+---------------+
|           ATL|       10600667|
|           ORD|        8037828|
|           DFW|        7894514|
|           DEN|        5863791|
|           LAX|        5709570|
|           IAH|        4697727|
|           PHX|        4548864|
|           SFO|        4171803|
|           LAS|        3910375|
|           MCO|        3357189|
+--------------+---------------+
only showing top 10 rows



In [0]:
# i) Show the airlines that make the maximum number of cancellations.
#from pyspark.sql.functions import col
#data2=plane_DF.groupBy('AIRLINE').agg({'CANCELLED' : 'sum'})
#data2.sort(col('sum(CANCELLED)').desc()).show()
max_cancellations=spark.sql("select AIRLINE,SUM(CANCELLED) as Total_cancellations from planetable group by AIRLINE order by SUM(CANCELLED) DESC")

max_cancellations.show()

+-------+-------------------+
|AIRLINE|Total_cancellations|
+-------+-------------------+
|     MQ|                414|
|     WN|                358|
|     EV|                312|
|     AA|                241|
|     DL|                177|
|     US|                169|
|     OO|                153|
|     B6|                145|
|     UA|                122|
|     NK|                 21|
|     VX|                 13|
|     AS|                 12|
|     F9|                 11|
|     HA|                  3|
+-------+-------------------+



In [0]:
# j) Find and order airlines in descending that make the most number of diversions.
#data3=plane_DF.groupBy('AIRLINE').agg({'DIVERTED' : 'sum'})
#data3.sort(col('sum(DIVERTED)').desc()).show()
spark.sql("select AIRLINE, SUM(DIVERTED) as Total_diversions from planetable group by AIRLINE order by Total_diversions desc").show()

+-------+----------------+
|AIRLINE|Total_diversions|
+-------+----------------+
|     WN|              35|
|     OO|              25|
|     EV|              22|
|     DL|              18|
|     B6|              16|
|     AA|              12|
|     US|               9|
|     UA|               8|
|     MQ|               5|
|     HA|               1|
|     NK|               0|
|     F9|               0|
|     AS|               0|
|     VX|               0|
+-------+----------------+



In [0]:
# k) Show days of month that seen the most number of diversion.
#data4=plane_DF.groupBy('DAY','MONTH').agg({'DIVERTED' : 'sum'})
#data4.sort(col('sum(DIVERTED)').desc()).show()
spark.sql("select DAY,MONTH,SUM(DIVERTED) as most_diverted from planetable group by DAY,MONTH order by most_diverted DESC").show()

+---+-----+-------------+
|DAY|MONTH|most_diverted|
+---+-----+-------------+
|  2|    2|            9|
| 14|    2|            7|
|  1|    3|            7|
|  5|    3|            7|
|  2|    3|            6|
|  4|    3|            6|
| 30|    1|            5|
|  7|    1|            5|
| 23|    2|            5|
| 18|    1|            5|
|  1|    2|            5|
| 11|    1|            5|
|  9|    2|            4|
|  8|    1|            4|
| 21|    2|            4|
| 12|    1|            3|
| 16|    2|            3|
|  9|    1|            3|
|  3|    3|            3|
| 31|    1|            3|
+---+-----+-------------+
only showing top 20 rows



In [0]:
# l) Calculate mean and standard deviation of departure delay for all flights in minutes.
spark.sql("select AIRLINE, AVG(DEPARTURE_DELAY) as Avg_DepDelay, STDDEV(DEPARTURE_DELAY) as std_dev_DepDelay from planetable group by AIRLINE ").show()

+-------+------------------+------------------+
|AIRLINE|      Avg_DepDelay|  std_dev_DepDelay|
+-------+------------------+------------------+
|     UA|14.293129770992367| 36.36679305829193|
|     NK|15.580739299610896|46.004490473829854|
|     AA|11.502293120638086| 50.58770621277648|
|     EV|11.534709861160556|  40.6022135127612|
|     B6|16.068664169787766|44.452210632841826|
|     DL| 9.938812083973374| 44.57836834851712|
|     OO|11.601510519690702|41.898251511636026|
|     F9|23.514687100893997| 55.21644901488807|
|     US| 7.806271591815041|29.947392168792227|
|     MQ| 17.07297384565709| 43.46879359686762|
|     HA|1.1835883171070931| 30.28223244986787|
|     AS|2.3365079365079366| 29.14656093746487|
|     VX| 9.857142857142858|35.175272138214424|
|     WN|10.115161630358397| 28.66211787470035|
+-------+------------------+------------------+



In [0]:
# m) Calculate mean and standard deviation of arrival delay for all flights in minutes.
spark.sql("select AIRLINE, AVG(ARRIVAL_DELAY) as Avg_ArrivalDelay, STDDEV(ARRIVAL_DELAY) as std_dev_ArrivalDelay from planetable group by AIRLINE ").show()

+-------+------------------+--------------------+
|AIRLINE|  Avg_ArrivalDelay|std_dev_ArrivalDelay|
+-------+------------------+--------------------+
|     UA| 6.697221614526362|   38.96583521422204|
|     NK|14.206426484907498|   47.57688184145277|
|     AA| 8.386631979187513|   53.56770860254512|
|     EV|10.884270870655678|   43.39199993434066|
|     B6| 13.95852534562212|   47.64386997495008|
|     DL|2.8144726712856043|   46.95730281739299|
|     OO|10.154792043399638|   43.76036682697078|
|     F9|24.103448275862068|   56.26804479748733|
|     US| 5.977315185481719|   34.10900474893441|
|     MQ|19.231592604605904|   46.39998574481305|
|     HA| 4.072423398328691|   32.38330048967069|
|     AS|-1.531766200762389|   31.37282637924461|
|     VX| 5.128571428571429|   40.86556219200457|
|     WN| 3.697840458351697|  31.225094725977485|
+-------+------------------+--------------------+



In [0]:
# n) Find all diverted Route from a source to destination Airport & which route is the most diverted.
spark.sql("select ORIGIN_AIRPORT,DESTINATION_AIRPORT, count(DIVERTED) as Total_Diversion from planetable group by ORIGIN_AIRPORT,DESTINATION_AIRPORT order by Total_Diversion desc").show(5)

+--------------+-------------------+---------------+
|ORIGIN_AIRPORT|DESTINATION_AIRPORT|Total_Diversion|
+--------------+-------------------+---------------+
|           LAX|                JFK|            146|
|           LAX|                SFO|            142|
|           JFK|                LAX|            137|
|           SFO|                LAX|            131|
|           LAS|                LAX|            123|
+--------------+-------------------+---------------+
only showing top 5 rows



In [0]:
# o) Finding AIRLINES with its total flight count, total number of flights arrival delayed by more than 30 Minutes, % of such flights delayed by more than 30 minutes when it is not Weekends with minimum count of flights from Airlines by more than 10. Also Exclude some of Airlines 'AK', 'HI', 'PR', 'VI' and arrange output in descending order by % of such count of flights.
